# Caras lab ephys analysis pipeline
This pipeline is intended to be run after extracting behavioral timestamps and neuron spike times with our [MatLab pipeline](https://github.com/caraslab/caraslab-spikesortingKS2)

Files need to be organized in a specific folder structure or file paths need to be changed

File structure can be found in the Sample dataset folder

## Imports
Specific imports can be found within each function

In [ ]:
%load_ext autoreload
%autoreload 2

from os import remove, makedirs
import warnings
from platform import system
from os.path import sep

from multiprocessing import Pool, cpu_count
from glob import glob

from helpers.run_ephys_pipeline import run_pipeline
from helpers.compile_fr_result_csv import compile_fr_result_csv
from matplotlib.pyplot import rcParams
from helpers.get_JSON_data import get_JSON_data
from helpers.PSTH_plotter_fromJSON import run_PSTH_pipeline
from helpers.auROC_heatmap_plotter import run_auROC_heatmap_pipeline

warnings.filterwarnings("ignore")

# Some plotting parameters
label_font_size = 11
tick_label_size = 7
legend_font_size = 6
line_thickness = 1

rcParams['figure.dpi'] = 600
rcParams['pdf.fonttype'] = 42
rcParams['ps.fonttype'] = 42
rcParams['font.family'] = 'Arial'
rcParams['font.weight'] = 'regular'
rcParams['axes.labelweight'] = 'regular'

rcParams['font.size'] = label_font_size
rcParams['axes.labelsize'] = label_font_size
rcParams['axes.titlesize'] = label_font_size
rcParams['axes.linewidth'] = line_thickness
rcParams['legend.fontsize'] = legend_font_size
rcParams['xtick.labelsize'] = tick_label_size
rcParams['ytick.labelsize'] = tick_label_size
rcParams['errorbar.capsize'] = label_font_size
rcParams['lines.markersize'] = line_thickness
rcParams['lines.linewidth'] = line_thickness

## Set global paths and variables

FILE NAMING REQUIREMENTS

This pipeline matches files using filenames.

If you make changes to the filenaming structures you will have to edit the filename matching code

Here are the defaults that come out of the MatLab processing pipeline:
- Synapse:
    - Behavior file: SUBJ-ID-154_MML-Aversive-AM-210501-112033_trialInfo.csv
    - Spike time file: SUBJ-ID-154_210501_concat_cluster2627.txt
- Intan:
    - Behavior file: SUBJ-ID-231_2021-07-17_15-19-28_Active_trialInfo.csv
    - Spike time file: SUBJ-ID-231_210717_concat_cluster651.txt

If you need to alter the filename matching structure, edit this function: helpers.preprocess_files.find_spoutfile_and_breakpoint

In [ ]:
DATA_PATH = '.' + sep + 'Sample_data'

SETTINGS_DICT = {
    'EXPERIMENT_TAG': 'OFCPL',  # Appends to start of summary files
    'SPIKES_PATH': DATA_PATH + sep + 'Spike times',
    'KEYS_PATH': DATA_PATH + sep + 'Key files',
    'OUTPUT_PATH': DATA_PATH + sep + 'Output',
    
    # If your data was concatenated before Kilosort
    'BREAKPOINT_PATH': DATA_PATH + sep + 'Breakpoints',
    
    # in seconds; for firing rate calculation to non-AM trials
    'NONAM_DURATION_FOR_FR': 1,  
    
    # in seconds; for firing rate calculation to AM trials; ignore shock artifact starting early
    'TRIAL_DURATION_FOR_FR': {'Hit': 0.95, 'FA': 0.95, 'Miss': 0.95},
    'AFTERTRIAL_FR_START': {'Hit': 1, 'FA': 1, 'Miss': 1.3},  # Shock artifact is ~0.3s long
    'AFTERTRIAL_FR_END': {'Hit': 1.9, 'FA': 1.9, 'Miss': 2.25},  # 0.95 duration to keep window consistent with trial
    
    'SHOCK_ARTIFACT': [0.95, 1.3],
    # These are for raw data extraction (no FR calculations)
    'PRETRIAL_DURATION_FOR_SPIKETIMES': 2,  # in seconds; for grabbing spiketimes around AM trials
    'POSTTRIAL_DURATION_FOR_SPIKETIMES': 5,  # in seconds; for grabbing spiketimes around AM trials
    
    'DEBUG_RUN': False,  # Turns off multiprocessing for debugging
    # For multiprocessing. Defaults to 4/5s of the number of cores
    'NUMBER_OF_CORES': 4 * cpu_count() // 5,

    # Only run these cells/subjects/sessions or None to run all
    'SESSIONS_TO_RUN': None,  # You can specify parts of the file name too
    'SESSIONS_TO_EXCLUDE': None,
    
    'OVERWRITE_PREVIOUS_CSV': True,  # False: appends to existing firing rate CSV file
    
    # Set up your recording platforms here
    'RECORDING_TYPE_DICT': {
        'SUBJ-ID-197': 'synapse',
        'SUBJ-ID-151': 'synapse',
        'SUBJ-ID-154': 'synapse',
        'SUBJ-ID-231': 'intan',
        # 'SUBJ-ID-232': 'intan',
        # 'SUBJ-ID-270': 'intan',
        'SUBJ-ID-389': 'intan',
        'SUBJ-ID-390': 'intan'
    },

    # Only needed for some older recordings processed by the MatLab pipeline. Newer files contain the sampling rates in them
    'SAMPLING_RATE_DICT': {
        'synapse': 24414.0625,
        'intan': 30000
    },
    
    # PSTH generation settings
    'PSTH_BIN_SIZE': 10,
    'PSTH_PRE_STIMULUS_DURATION': 2,
    'PSTH_POST_STIMULUS_DURATION': 4,
    'PSTH_FIXED_YLIM': 60,
    'PSTH_RASTER_YLIM': 30.5,
    'PSTH_ALIGN_TO_RESPONSE': False,
    'PSTH_TRIALTYPES': [
        'Hit (shock)',  # Trials above threshold
        # 'Hit (no shock)', # Trials below threshold
        'False alarm',
        'Miss (shock)',  # Trials above threshold
        # 'Miss (no shock)', # Trials below threshold
        'Passive',
    ],
    
    # AuROC heatmap generation settings
    'AUROC_GROUPING_FILE': DATA_PATH + sep + 'Output' + sep + 'allUnits_list.csv',
    'AUROC_GROUPING_VARIABLE': 'ActiveBaseline_modulation_direction',
    'AUROC_UNIQUE_GROUPS': ['decrease', 'increase', 'none'],
    'AUROC_GROUP_COLORS': ['#E49E50', '#5AB4E5', '#939598'],
    'AUROC_TRIALTYPES': {
        # Use these if you would like to sort based on trial (AM)-aligned auROC
        'PassivePre': [0, 1.5],
        'Hit_shockOn_auroc': [0, 1.5],
        # 'Hit_shockOff_auroc': [0, 1.5],
        'FA_auroc': [0, 1.5],
        'Miss_shockOn_auroc': [1.4, 2.9],
        # 'Miss_shockOff_auroc': [1.4, 2.9]
        'PassivePost': [0, 1.5],
        
        # Use these if you would like to sort based on response (spout offset)-aligned auROC
        # 'SpoutOff_hits_auroc': [-0.5, 0],
        # 'SpoutOff_FAs_auroc': [-0.5, 0],
        # 'SpoutOff_misses_auroc': [-0.5, 0]
    },
    
    'SORT_BY_WHICH_TRIALTYPE': 'Hit_shockOn_auroc',
    
    'AUROC_BIN_SIZE': 0.1,
    'AUROC_PRE_STIMULUS_DURATION': 2,
    
    'AUROC_POST_STIMULUS_DURATION': 4,
    
    # Below is a switchboard of functions you desire to run from the pipeline
    # If you change your mind later, you can just run the ones you want and the code will add it to existing JSON files
    'PIPELINE_SWITCHBOARD': {
        # Trial-by-trial firing rates
        'firing_rate_to_trials': True,  
        
        # auROC analyses need by definition to combine trials, so each of these pool a specific type of trial and align to specific events
        # Trials aligned by AM onset
        'auROC_hit': True,
        'auROC_FA': True,
        'auROC_miss': True,
        'auROC_hitByShock': True,  # Separate trials by whether a shock was going to be delivered
        'auROC_missByShock': True, # Separate trials by whether a shock was delivered
        'auROC_AMTrial': True,  # Agnostic of whether miss or hit
        'auROC_AMdepthByAMdepth': False, # Agnostic of whether miss or hit, but separated by AM depth
        
        # Trials aligned by spoutOff events
        'auROC_spoutOffHit': True,
        'auROC_spoutOffFA': True,
        'auROC_spoutOffMiss': True,
        'auROC_spoutOffHitByShock': True,  # Separate trials by whether a shock was going to be delivered
        'auROC_spoutOffMissByShock': True, # Separate trials by whether a shock was delivered
        
        # PSTH plotting
        'plot_trialType_PSTH': True,
        'plot_AMDepth_PSTH': True
    }
}


## Initial file matching then run the pipeline
Uses multiprocessing to process many units at once

In [ ]:
makedirs(SETTINGS_DICT['OUTPUT_PATH'] + sep + 'JSON files', exist_ok=True)

# Load existing JSONs; will be empty if this is the first time running
json_filenames = glob(SETTINGS_DICT['OUTPUT_PATH'] + sep + 'JSON files' + sep + '*json')

# Clear older temp files if they exist
process_tempfiles = glob(SETTINGS_DICT['OUTPUT_PATH'] + sep + '*_tempfile_*.csv')
[remove(f) for f in process_tempfiles]

# Generate a list of inputs to be passed to each worker
input_lists = list()
memory_paths = glob(SETTINGS_DICT['SPIKES_PATH'] + sep + '*cluster*.txt')

for dummy_idx, memory_path in enumerate(memory_paths):
    if SETTINGS_DICT['SESSIONS_TO_RUN'] is not None:
        if any([chosen for chosen in SETTINGS_DICT['SESSIONS_TO_RUN'] if chosen in memory_path]):
            pass
        else:
            continue
            
    if SETTINGS_DICT['SESSIONS_TO_EXCLUDE'] is not None:
        if any([chosen for chosen in SETTINGS_DICT['SESSIONS_TO_EXCLUDE'] if chosen in memory_path]):
            continue
        else:
            pass

    if SETTINGS_DICT['DEBUG_RUN']:
        run_pipeline((memory_path, json_filenames, SETTINGS_DICT))
    else:
        input_lists.append((memory_path, json_filenames, SETTINGS_DICT))

if not SETTINGS_DICT['DEBUG_RUN']:
    pool = Pool(SETTINGS_DICT['NUMBER_OF_CORES'])

    # # Feed each worker with all memory paths from one unit
    pool_map_result = pool.map(run_pipeline, input_lists)

    pool.close()

    pool.join()
    
    compile_fr_result_csv(SETTINGS_DICT['EXPERIMENT_TAG'], SETTINGS_DICT['OUTPUT_PATH'], SETTINGS_DICT['OVERWRITE_PREVIOUS_CSV'])

## Now we can use the JSONs exported from the code above for a faster exploration of the data

In [ ]:
# Sessions you want to run; edit the specifics in helpers.get_JSON_data if you want to change
session_tags = ['pre', 'active', 'post', 'post1h']

# Load existing JSONs
json_filenames = glob(SETTINGS_DICT['OUTPUT_PATH'] + sep + 'JSON files' + sep + '*json')

# Retrieve data in JSON
data_dict = dict()
print("Loading data in JSONs...")
for session_type in session_tags:
    unit_list, data_list = get_JSON_data(json_filenames, session_type, 
                                                  sessions_to_run=SETTINGS_DICT['SESSIONS_TO_RUN'], 
                                                  sessions_to_exclude=SETTINGS_DICT['SESSIONS_TO_EXCLUDE'])
    for unit_idx, unit in enumerate(unit_list):
        if unit not in data_dict.keys():
            data_dict.update({unit: {}})
        data_dict[unit].update({session_type: data_list[unit_idx]})

## PSTH plotting
Unsophisticated PSTH plotting engine. Need to make tweaks if you want something fancier
Uses multiprocessing to process many units at once

In [46]:
if not SETTINGS_DICT['DEBUG_RUN']:
    pool = Pool(SETTINGS_DICT['NUMBER_OF_CORES'])

    # Feed each worker one unit_name
    input_list = [(unit_name, data_dict[unit_name], SETTINGS_DICT) for unit_name in data_dict.keys()]
    pool_map_result = pool.map(run_PSTH_pipeline, input_list)

    pool.close()

    pool.join()
else:
    [run_PSTH_pipeline((unit_name, data_dict[unit_name], SETTINGS_DICT)) for unit_name in data_dict.keys()]

KeyboardInterrupt: 

# AuROC heatmaps
Quick summary of what auROC profiles look like across the entire population.

In [43]:
run_auROC_heatmap_pipeline(data_dict, SETTINGS_DICT)